In [1]:
import sys
sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/AudioClassification-CoughVID')
import os
import yaml
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchaudio
from models.conv_vae import ConvVAE
from readers.coughvid_reader import CoughVID_NormalAnomaly, CoughVID_Dataset
from readers.featurizer import Wave2Mel
from readers.collate_fn import collate_fn

## 读取数据集

In [3]:
healthy_p_list, healthy_label_list, unhealthy_p_list, unhealthy_label_list = CoughVID_NormalAnomaly()
print(len(healthy_p_list), len(unhealthy_p_list))
print(len(healthy_label_list), len(unhealthy_label_list))
# 训练集取健康样本的13000个，剩下503个和非健康样本的503个一起组成验证集，测试集就是验证集。
trp, trl, vap, val = healthy_p_list[:13000], healthy_label_list[:13000], healthy_p_list[13000:], healthy_label_list[13000:]
vap.extend(unhealthy_p_list[:503])
val.extend(unhealthy_label_list[:503])
print(len(trp), len(trl))
print(len(vap), len(val))

13503 4173
13503 4173
13000 13000
1006 1006


## 最耗时的读取

In [4]:
tic = time.time()
train_dataset = CoughVID_Dataset(path_list=trp, label_list=trl)
toc = time.time()
print("Train Dataset Creat Completely, cost time:", toc-tic)
valid_dataset = CoughVID_Dataset(path_list=vap, label_list=val)
toc = time.time()
print("Valid Dataset Creat Completely, cost time:", toc-tic)

Loading:   0%|                                                                               | 0/13000 [00:00<?, ?it/s]C:\Program Files (zk)\PythonFiles\AClassification\SoundDL-CoughVID\readers\audio.py:120: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, sample_rate = librosa.load(file)  # , dtype='float32')
C:\Users\zhaoke\miniconda3\envs\torch-0\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Loading:   0%|                                                                     | 1/13000 [00:01<3:46:34,  1.05s/it]C:\Program Files (zk)\PythonFiles\AClassification\SoundDL-CoughVID\readers\audio.py:120: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, sample_rate = librosa.load(file)  # , dtype='float32')
C:\Users\zhaoke\miniconda3\envs\torch-0\lib\sit

Train Dataset Creat Completely, cost time: 926.6890480518341


Loading: 100%|█████████████████████████████████████████████████████████████████████| 1006/1006 [01:12<00:00, 13.85it/s]

Valid Dataset Creat Completely, cost time: 999.3521203994751


## 基本参数配置

In [5]:
configs = {
    "run_save_dir": "./runs/tdnn_coughvid/",
    "model":{
        "num_class": 3,
        "input_length": 94,
        "wav_length": 48000,
        "input_dim": 512,
        "n_mels": 80,
        },
    "fit":{
        "batch_size": 64,
        "epochs" : 23,
        "start_scheduler_epoch": 6
        },
}

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
num_epoch = configs["fit"]["epochs"]
klw = 0.00025
# istrain: 如果是评估环节，设为False，读取测试集，并且不创建optimizer
# isdemo: 如果只是测试一下，设为True，仅读取32条数据方便快速测试是否有bug
istrain, isdemo = True, False

In [ ]:
# model loss_function optimizer scheduler
print("All model and loss are on device:", device)
model = ConvVAE(shape=(1, 94, 128), flat=True).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=5e-5)
print("Create TDNN, Adam with lr=1e-3, CosineAnnealingLR Shceduler")

MSE_loss = nn.MSELoss(reduction="mean")
def vae_loss(X, X_hat, mean, logvar, kl_weight=0.0001):
    reconstruction_loss = MSE_loss(X_hat, X)
    KL_divergence = 0.5 * torch.sum(-1 - logvar + torch.exp(logvar) + mean ** 2)
    # print(reconstruction_loss.item(), KL_divergence.item())
    return reconstruction_loss + kl_weight * KL_divergence

print("Create VAELoss...")

w2m = Wave2Mel(sr=16000)

### 测试模型正确性

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
m = ConvVAE(shape=(1, 94, 128), flat=True).to(device)
x = torch.randn(size=(16, 1, 94, 128)).to(device)

# print(m(x).shape)
recon_mel, z, mean_lant, logvar_lant = m(x)  # (28-3+2p)/2+1
print(recon_mel.shape)
print(mean_lant)

torch.Size([16, 1, 94, 128])
tensor([[-0.2590, -0.3112, -0.4815,  ...,  0.3014, -0.0775, -0.0698],
        [ 0.1674, -0.8358, -0.0267,  ..., -0.3281, -0.1714,  0.5831],
        [ 0.4262, -0.1466, -0.8161,  ..., -0.2738, -0.2146,  0.6505],
        ...,
        [ 0.1519,  0.1511,  0.0544,  ...,  0.0386, -0.3849, -0.2341],
        [-0.5821, -0.0683, -0.1159,  ..., -0.4339,  0.3537, -0.3093],
        [-0.1273, -0.5743, -0.8791,  ..., -0.9140, -0.0521,  0.0253]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,
                              collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True,
                              collate_fn=collate_fn)
print("Create Training Loader and Valid Loader.")

Create Training Loader and Valid Loader.


## 运行保存目录

In [11]:
timestr = time.strftime("%Y%m%d%H%M", time.localtime())
if istrain:
    run_save_dir = configs["run_save_dir"] + timestr + f'_tdnn_focalloss/'
if not isdemo:
    os.makedirs(run_save_dir, exist_ok=True)
    print("创建运行保存文件", run_save_dir)

创建运行保存文件 ./runs/tdnn_coughvid/202404261420_tdnn_focalloss/


In [14]:
history1 = []
for epoch_id in range(configs["fit"]["epochs"]):
    # ---------------------------
    # -----------TRAIN-----------
    # ---------------------------
    model.train()
    for x_idx, (x_wav, y_label, _) in enumerate(tqdm(train_loader, desc="Training")):
        x_mel = w2m(x_wav).transpose(1,2).unsqueeze(1).to(device)
        y_label = torch.tensor(y_label, device=device)
        # print("shape of x_mel:", x_mel.shape)
        optimizer.zero_grad()
        recon_mel, _, mean_latent, logvar_latent = model(x=x_mel)
        # recon_loss = self.recon_loss(recon_spec, x_mel)
        pred_loss = vae_loss(x_mel, recon_mel, mean_latent, logvar_latent)
        pred_loss.backward()
        optimizer.step()

        if x_idx > 2:
            history1.append(pred_loss.item())
        if x_idx % 60 == 0:
            print(f"Epoch[{epoch_id}], mtid pred loss:{pred_loss.item():.4f}")
    if epoch_id >= configs["fit"]["start_scheduler_epoch"]:
        scheduler.step()

    # ---------------------------
    # -----------SAVE------------
    # ---------------------------
    plt.figure(0)
    plt.plot(range(len(history1)), history1, c="green", alpha=0.7)
    plt.savefig(run_save_dir + f'cls_loss_iter_{epoch_id}.png')
    plt.close()
    # if epoch > 6 and epoch % 2 == 0:
    os.makedirs(run_save_dir + f"model_epoch_{epoch_id}/", exist_ok=True)
    tmp_model_path = "{model}model_{epoch}.pth".format(
        model=run_save_dir + f"model_epoch_{epoch_id}/",
        epoch=epoch_id)
    torch.save(model.state_dict(), tmp_model_path)
    # ---------------------------
    # -----------TEST------------
    # ---------------------------
    # model.eval()
    # heatmap_input = None
    # labels = None
    # for x_idx, (x_wav, y_label, _) in enumerate(tqdm(valid_loader, desc="Validate")):
    #     x_mel = w2m(x_wav).transpose(1,2).unsqueeze(1).to(device)
    #     y_label = torch.tensor(y_label, device=device)
    #     recon_mel, _, mean_latent, logvar_latent = model(x=x_mel)
    #     # recon_loss = self.recon_loss(recon_spec, x_mel)
    #     pred_loss = vae_loss(x_mel, recon_mel, mean_latent, logvar_latent)
    #     if x_idx == 0:
    #         heatmap_input, labels = y_pred, y_label
    #     else:
    #         heatmap_input = torch.concat((heatmap_input, y_pred), dim=0)
    #         labels = torch.concat((labels, y_label), dim=0)
    #     # if x_idx * configs["fit"]["batch_size"] > 800:
    #     #     break
    # print("heatmap_input shape:", heatmap_input.shape)
    # print("lables shape:", labels.shape)
    # # if epoch > 3:
    # #     self.plot_reduction(resume_path="", load_epoch=epoch, reducers=["heatmap"])
    # heatmap_input = heatmap_input.detach().cpu().numpy()
    # labels = labels.detach().cpu().numpy()
    # calc_accuracy(pred_matrix=heatmap_input, label_vec=labels,
    #               save_path=run_save_dir + f"/accuracy_epoch_{epoch_id}.txt")
    # plot_heatmap(pred_matrix=heatmap_input, label_vec=labels,
    #              ticks=["healthy", "symptomatic", "COVID-19"],
    #              save_path=run_save_dir + f"/heatmap_epoch_{epoch_id}.png")
print("============== END TRAINING ==============")

Training:   0%|                                                                                | 0/407 [00:00<?, ?it/s]C:\Users\zhaoke\AppData\Local\Temp\ipykernel_30868\2723303609.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_label = torch.tensor(y_label, device=device)
Training:   0%|▎                                                                       | 2/407 [00:00<00:20, 19.32it/s]

Epoch[0], mtid pred loss:253.6730


Training:  16%|███████████▏                                                           | 64/407 [00:02<00:13, 24.51it/s]

Epoch[0], mtid pred loss:223.1687


Training:  30%|█████████████████████▎                                                | 124/407 [00:05<00:12, 21.82it/s]

...

Epoch[22], mtid pred loss:158.1978


Training:  89%|██████████████████████████████████████████████████████████████▎       | 362/407 [00:15<00:01, 23.97it/s]

Epoch[22], mtid pred loss:157.0493


Training: 100%|██████████████████████████████████████████████████████████████████████| 407/407 [00:17<00:00, 23.17it/s]


============== END TRAINING ==============


# End